In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#https://api.tidesandcurrents.noaa.gov/api/prod/

url = 'https://api.tidesandcurrents.noaa.gov/api/prod/'
parameters = 'datagetter?date=latest&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=json'

r = requests.get(url + parameters)

r.status_code

200

In [3]:
r.json()

{'metadata': {'id': '9410170',
  'name': 'San Diego',
  'lat': '32.7156',
  'lon': '-117.1767'},
 'data': [{'t': '2025-01-12 21:53', 'v': '7.927'}]}

In [4]:
parameters = 'datagetter?begin_date=20250110&end_date=20250112&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=csv'
r = requests.get(url + parameters)

r.status_code

200

In [5]:
csv_file = open('test.csv', 'wb')
csv_file.write(r.content)
csv_file.close()

In [ ]:
paramters = 'datagetter?date=latest&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=json'


### Use Selenium for dynamic rendered websites

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [2]:
#use the path you saved it in
PATH = r"C:\Program Files\chromedriver-win64\chromedriver.exe"

URL = "https://tidesandcurrents.noaa.gov/stations.html?type=Water+Levels"

In [3]:
service = Service(executable_path=PATH)

driver = webdriver.Chrome(service=service)

In [4]:
STATION_ID = [] #grab all stations that take water level measurements


In [ ]:
driver.get(URL)

wait = WebDriverWait(driver, 10)  # stay on site for 10s

a_elements = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, '//a[@style="color: #015FA9;"]'))
)

In [ ]:
for element in a_elements:
    # regex for extracting 7-digit station id
    match = re.search(r'waterlevels\.html\?id=(\d{7})', element.get_attribute('href'))
    if match:
        STATION_ID.append(match.group(1))

In [14]:
print(len(STATION_ID)) #test if correctly collected
print(STATION_ID[0])
print(STATION_ID[len(STATION_ID) - 1])


292
8735180
9449880


In [11]:
driver.quit()